In [47]:
import numpy as np
from numpy import asarray

In [48]:
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
from os import listdir

In [ ]:
damagepath = "/code/Project2/COE379L-Project2/damage"
nodamagepath = "/code/Project2/COE379L-Project2/no_damage"

# Create independent and dependent variables as a dictionary
X = []
y = []
for images in os.listdir(damagepath):
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        img_path = os.path.join(damagepath, images)
        img = load_img(img_path)
        imagedata = img_to_array(img)
        X.append(imagedata)
        y.append(1)

for images in os.listdir(nodamagepath):
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        img_path = os.path.join(nodamagepath, images)
        img = load_img(img_path)
        imagedata = img_to_array(img) 
        X.append(imagedata)
        y.append(0)
X = np.array(X)  
y = np.array(y)

In [ ]:
print("Total images:", X.shape[0])
print("Total labels:", y.shape[0])